In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fredapi as fa

In [2]:
#pull data from FRED
fred = fa.Fred(api_key='b6bbb1bfe76d93bf0ca60a69b99d3474')

#CPI
cpi = fred.get_series('CPIAUCSL')
cpi.name = 'CPI'

cpi_data = pd.DataFrame(index=cpi.index,data=cpi.values,columns=['CPI']).dropna()
cpi_data['CPI YOY'] = cpi_data['CPI'].rolling(12).apply(lambda x: (x.iloc[-1]/x.iloc[0] - 1) *100 )
cpi_data['CPI'] = cpi_data['CPI YOY'].rolling(12).apply(lambda x: x.iloc[-1] -x.iloc[0] - 1)
cpi_data = cpi_data.dropna()

#GDP
gdp = fred.get_series('GDP')
gdp.name = 'GDP'

gdp_data = pd.DataFrame(index=gdp.index,data=gdp.values,columns=['GDP']).dropna()
gdp_data['GDP YOY'] = gdp_data['GDP'].rolling(4).apply(lambda x: (x.iloc[-1]/x.iloc[0] - 1) *100 )
gdp_data['GDP'] = gdp_data['GDP YOY'].rolling(4).apply(lambda x: x.iloc[-1] -x.iloc[0] - 1)
gdp_data = gdp_data.dropna()

#merge data
df = pd.concat([gdp_data, cpi_data], axis=1).dropna()

#make cycle
conditions = [
    (df['CPI'] < 0.0) & (df['GDP'] >= 0.0),
    (df['CPI'] >= 0.0) & (df['GDP'] >= 0.0), #& (df['GDP YOY'] > 0.5),
    (df['CPI'] < 0.0) & (df['GDP'] < 0.0)]
choices = [1.0, 2.0, 4.0]
df['cycle_gdp_cpi'] = np.select(conditions, choices, default=3.0)
df

,GDP,GDP YOY,CPI,CPI YOY,cycle_gdp_cpi
1949-01-01,-9.302987,0.905099,-7.154492,1.436417,4.0
1949-04-01,-11.298352,-2.809854,-10.115786,-0.374844,4.0
1949-07-01,-9.169953,-2.666871,-10.949600,-2.988129,4.0
1949-10-01,-3.507446,-1.602347,-6.231905,-2.028146,4.0
1950-01-01,5.302378,3.492524,-4.417621,-1.672940,1.0
...,...,...,...,...,...
2021-04-01,8.470623,7.885368,2.984727,4.213031,2.0
2021-07-01,6.729891,8.503750,2.692618,4.855536,2.0
2021-10-01,-5.512110,9.121111,4.134840,6.086583,3.0
2022-01-01,-1.537119,7.348249,4.051159,7.058015,3.0


In [3]:
#HY Cycle builder
hyoas = fred.get_series('BAMLH0A0HYM2')
hyoas.name = 'HY_OAS'

data = pd.DataFrame(index=hyoas.index,data=hyoas.values,columns=['HY OAS']).dropna()

#ten year median
median_10y = data['HY OAS'].rolling('3650D').median()

#3 month rising or falling: 1 is rising
return_3m = (data['HY OAS'].pct_change()+1).rolling('91D').apply(np.prod)-1
return_3m = pd.DataFrame(index= return_3m.index, data= np.where(return_3m > 0, 1,0))

#add to data frame
data['10y Median'] = median_10y
data['3M Up/Down'] = return_3m

#make HY cycle
conditions = [
    (data['HY OAS']>data['10y Median']) & (data['3M Up/Down']==0),
    (data['HY OAS']<data['10y Median']) & (data['3M Up/Down']==0),
    (data['HY OAS']<data['10y Median']) & (data['3M Up/Down']==1)
]
choices = [1,2,3]

data['cycle_hy'] = np.select(conditions, choices, default = 4)

#convert to qtrly to match GDP - may want both in monthly
data = data.resample('Q', label='left').mean() #may want to round
data = data.reset_index()
data['date'] = data['index'] + pd.DateOffset(1)
data = data[['date','cycle_hy']]
data.set_index('date',inplace=True)
#data

In [4]:
#get recession info
nber = fred.get_series('USREC')
nber.name = 'contraction'

nber = pd.DataFrame(index=nber.index,data=nber.values,columns=['contraction']).dropna().resample('Q', label='left').mean().apply(np.ceil)
nber = nber.reset_index()
nber['date'] = nber['index'] + pd.DateOffset(1)
nber = nber[['date','contraction']]
nber.set_index('date',inplace=True)
#nber

In [9]:
#combine both HY and GDP
model_df = pd.concat([data, df], axis=1)
#get the mean of the two
model_df['cycle_avg'] = model_df[['cycle_gdp_cpi','cycle_hy']].mean(axis=1)
model_df = model_df[['cycle_avg','cycle_hy','cycle_gdp_cpi']].fillna('-')
#combine recession data
model_df = pd.concat([nber, model_df], axis=1).dropna()

#model in the recession 

#options for modeling without recession adjustments
model_df['cycle_model'] = np.where((model_df['cycle_avg'] >= 3.5) & (model_df['contraction']==0), 3, model_df['cycle_avg'])
model_df['cycle_modelv2'] = np.where((model_df['cycle_avg'] >= 2.5) & (model_df['contraction']==0), model_df['cycle_avg']-0.5, model_df['cycle_avg'])
#model_df['cycle_modelv3'] = np.where((model_df['cycle_avg'] > 3.5) & (model_df['contraction']==0), 2, model_df['cycle_avg'])

#options for modeling WITH recession adjustments
model_df['cycle_model'] = np.where((model_df['cycle_avg'] < 3) & (model_df['contraction']==1), 3, model_df['cycle_model']).round()
model_df['cycle_modelv2'] = np.where((model_df['cycle_avg'] < 3) & (model_df['contraction']==1), model_df['cycle_modelv2']+0.5, model_df['cycle_modelv2']).round()
#model_df['cycle_modelv3'] = np.where((model_df['cycle_avg'] < 3) & (model_df['contraction']==1), 3, model_df['cycle_avg']).round()

model_df = model_df[['cycle_model','cycle_modelv2','cycle_avg','cycle_hy','cycle_gdp_cpi','contraction']]
model_df

,cycle_model,cycle_modelv2,cycle_avg,cycle_hy,cycle_gdp_cpi,contraction
1949-01-01,4.0,4.0,4.000000,-,4.0,1.0
1949-04-01,4.0,4.0,4.000000,-,4.0,1.0
1949-07-01,4.0,4.0,4.000000,-,4.0,1.0
1949-10-01,4.0,4.0,4.000000,-,4.0,1.0
1950-01-01,1.0,1.0,1.000000,-,1.0,0.0
...,...,...,...,...,...,...
2021-07-01,2.0,2.0,2.179104,2.358209,2.0,0.0
2021-10-01,3.0,2.0,2.742424,2.484848,3.0,0.0
2022-01-01,3.0,2.0,2.921875,2.84375,3.0,0.0
2022-04-01,3.0,3.0,3.276923,3.553846,3.0,0.0


Returns from the factors

In [11]:
facts = pd.read_csv("Factors.csv",header=3)
facts = facts.drop("String",axis=1)
facts['Date'] = pd.to_datetime(facts['Date'])
facts = facts.set_index("Date")
facts = facts.apply(pd.to_numeric)/100

#resample to qtrs to match the
facts += 1
facts = facts.rolling(3).apply(lambda x: x.prod()-1).dropna()
facts

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1963-09-01,0.030171,-0.017204,0.009436,0.003250,-0.012403,0.007921,0.021127
1963-10-01,0.060369,-0.026875,0.018304,0.024376,-0.020698,0.008122,0.043594
1963-11-01,0.000625,-0.027660,0.017804,0.015496,0.004755,0.008323,0.025514
1963-12-01,0.035188,-0.043104,0.016279,0.023064,0.001148,0.008524,0.041482
1964-01-01,0.032260,-0.028354,0.032352,-0.003110,0.036703,0.008625,0.018656
...,...,...,...,...,...,...,...
2022-04-22,-0.088351,0.003434,0.074486,-0.001085,0.126981,0.000200,0.099381
2022-05-22,-0.070158,-0.025999,0.130484,0.034824,0.136269,0.000500,0.107160
2022-06-22,-0.173744,0.008343,0.082479,0.070670,0.049592,0.001000,0.083405


In [90]:
#merge to get model and return)
returns = pd.concat([facts, model_df], axis=1).dropna()
returns = returns[['Mkt-RF','SMB','HML','RMW','CMA','Mom','cycle_model','cycle_modelv2','cycle_hy','cycle_gdp_cpi']]
returns

,Mkt-RF,SMB,HML,RMW,CMA,Mom,cycle_model,cycle_modelv2,cycle_hy,cycle_gdp_cpi
1963-10-01,0.060369,-0.026875,0.018304,0.024376,-0.020698,0.043594,1.0,1.0,-,1.0
1964-01-01,0.032260,-0.028354,0.032352,-0.003110,0.036703,0.018656,1.0,1.0,-,1.0
1964-04-01,0.030747,-0.000296,0.055933,-0.035002,0.030344,0.004261,3.0,4.0,-,4.0
1964-07-01,0.044952,0.001565,0.032602,-0.003599,0.019097,0.026812,3.0,4.0,-,4.0
1964-10-01,0.018084,0.009392,0.029722,-0.008688,0.014467,-0.005496,3.0,4.0,-,4.0
...,...,...,...,...,...,...,...,...,...,...
2000-07-01,-0.024955,0.046199,0.039404,0.010613,0.012327,0.060763,3.0,3.0,4.0,3.0
2000-10-01,-0.015962,-0.034695,0.117154,0.087863,0.107555,0.029829,3.0,4.0,4.0,4.0
2022-01-01,-0.068298,0.082857,0.147154,0.096285,0.078925,-0.221151,3.0,2.0,2.84375,3.0
2022-04-01,-0.099569,0.043109,0.140992,0.093139,0.096991,0.125414,3.0,3.0,3.553846,3.0


In [91]:
def print_describe(model):
    cycle_df = returns[['Mkt-RF','SMB','HML','RMW','CMA','Mom',model]].apply(pd.to_numeric, errors='coerce').dropna()
    cycle_df[model] = cycle_df[model].round()
    for x in range(1,5):
        cycle = cycle_df.loc[cycle_df[model]==x]
        count = cycle_df.loc[cycle_df[model]==x].describe()[model][0]
        print("Descriptive Stats during cycle {} which is {}/{}".format(x, count, len(cycle_df)))
        print(cycle.describe()[['Mkt-RF','SMB','HML','RMW','CMA','Mom']].round(4))
        

#### model 1 descriptive stats

In [92]:
print_describe('cycle_model')

Descriptive Stats during cycle 1 which is 33.0/152
        Mkt-RF      SMB      HML      RMW      CMA      Mom
count  33.0000  33.0000  33.0000  33.0000  33.0000  33.0000
mean    0.0253   0.0183   0.0170   0.0049   0.0063   0.0218
std     0.0615   0.0591   0.0501   0.0305   0.0341   0.0476
min    -0.0839  -0.0803  -0.1157  -0.0490  -0.1102  -0.0973
25%    -0.0159  -0.0313  -0.0155  -0.0160  -0.0060  -0.0102
50%     0.0179   0.0213   0.0170   0.0029   0.0084   0.0145
75%     0.0557   0.0658   0.0500   0.0217   0.0354   0.0470
max     0.1658   0.1304   0.1271   0.0673   0.0544   0.1357
Descriptive Stats during cycle 2 which is 13.0/152
        Mkt-RF      SMB      HML      RMW      CMA      Mom
count  13.0000  13.0000  13.0000  13.0000  13.0000  13.0000
mean   -0.0110  -0.0118   0.0252  -0.0007   0.0135   0.0090
std     0.0905   0.0547   0.0331   0.0327   0.0249   0.0702
min    -0.2260  -0.0887  -0.0151  -0.0763  -0.0238  -0.1028
25%    -0.0631  -0.0513  -0.0046  -0.0216  -0.0092  -0.029

Descriptive Stats of 2nd Model

In [88]:
print_describe('cycle_modelv2')

Descriptive Stats during cycle 1 which is 33.0/152
        Mkt-RF      SMB      HML      RMW      CMA      Mom
count  33.0000  33.0000  33.0000  33.0000  33.0000  33.0000
mean    0.0253   0.0183   0.0170   0.0049   0.0063   0.0218
std     0.0615   0.0591   0.0501   0.0305   0.0341   0.0476
min    -0.0839  -0.0803  -0.1157  -0.0490  -0.1102  -0.0973
25%    -0.0159  -0.0313  -0.0155  -0.0160  -0.0060  -0.0102
50%     0.0179   0.0213   0.0170   0.0029   0.0084   0.0145
75%     0.0557   0.0658   0.0500   0.0217   0.0354   0.0470
max     0.1658   0.1304   0.1271   0.0673   0.0544   0.1357
Descriptive Stats during cycle 2 which is 34.0/152
        Mkt-RF      SMB      HML      RMW      CMA      Mom
count  34.0000  34.0000  34.0000  34.0000  34.0000  34.0000
mean   -0.0082   0.0050   0.0069   0.0025   0.0090   0.0204
std     0.0677   0.0616   0.0566   0.0501   0.0341   0.0826
min    -0.2260  -0.1040  -0.1557  -0.1713  -0.0685  -0.2212
25%    -0.0520  -0.0468  -0.0208  -0.0258  -0.0148  -0.010

### Just HY Model

In [89]:
print_describe('cycle_hy')

Descriptive Stats during cycle 1 which is 2.0/20
       Mkt-RF     SMB     HML     RMW     CMA     Mom
count  2.0000  2.0000  2.0000  2.0000  2.0000  2.0000
mean   0.1005 -0.0335 -0.0500 -0.0598 -0.0375  0.0164
std    0.0923  0.0251  0.0929  0.0233  0.1029  0.1687
min    0.0353 -0.0513 -0.1157 -0.0763 -0.1102 -0.1028
25%    0.0679 -0.0424 -0.0828 -0.0681 -0.0738 -0.0432
50%    0.1005 -0.0335 -0.0500 -0.0598 -0.0375  0.0164
75%    0.1331 -0.0246 -0.0172 -0.0516 -0.0011  0.0760
max    0.1658 -0.0158  0.0157 -0.0434  0.0353  0.1357
Descriptive Stats during cycle 2 which is 4.0/20
       Mkt-RF     SMB     HML     RMW     CMA     Mom
count  4.0000  4.0000  4.0000  4.0000  4.0000  4.0000
mean   0.0833  0.0212 -0.0183 -0.0233 -0.0065  0.0664
std    0.0869  0.1066  0.1038  0.1009  0.0408  0.1027
min   -0.0168 -0.0854 -0.1557 -0.1713 -0.0483  0.0017
25%    0.0413 -0.0400 -0.0566 -0.0458 -0.0337  0.0077
50%    0.0786  0.0029 -0.0055  0.0166 -0.0110  0.0225
75%    0.1206  0.0641  0.0328  0.0391 

### Just GDP and Inflation

In [93]:
print_describe('cycle_gdp_cpi')

Descriptive Stats during cycle 1 which is 36.0/151
        Mkt-RF      SMB      HML      RMW      CMA      Mom
count  36.0000  36.0000  36.0000  36.0000  36.0000  36.0000
mean    0.0248   0.0145   0.0184   0.0046   0.0060   0.0207
std     0.0608   0.0582   0.0484   0.0297   0.0334   0.0476
min    -0.0839  -0.0803  -0.1157  -0.0490  -0.1102  -0.0973
25%    -0.0169  -0.0313  -0.0092  -0.0174  -0.0148  -0.0106
50%     0.0175   0.0097   0.0176   0.0023   0.0077   0.0138
75%     0.0569   0.0647   0.0483   0.0221   0.0353   0.0481
max     0.1658   0.1304   0.1271   0.0673   0.0544   0.1357
Descriptive Stats during cycle 2 which is 11.0/151
        Mkt-RF      SMB      HML      RMW      CMA      Mom
count  11.0000  11.0000  11.0000  11.0000  11.0000  11.0000
mean   -0.0316  -0.0071   0.0206   0.0016   0.0160   0.0194
std     0.0921   0.0584   0.0381   0.0289   0.0272   0.0675
min    -0.2260  -0.0887  -0.0233  -0.0491  -0.0238  -0.0883
25%    -0.0853  -0.0553  -0.0078  -0.0151  -0.0068  -0.013